In [ ]:
#| default_exp migrate

# migrate
> Utilities for migrating to nbdev
- order: 16

In [ ]:
#| export
from nbdev.process import *
from nbdev.frontmatter import *
from nbdev.frontmatter import _fm2dict, _re_fm_md, _dict2fm, _insertfm
from nbdev.processors import *
from nbdev.config import get_config, read_nb, set_version, pyproject_tmpl, nbdev_defaults
from nbdev.sync import write_nb
from nbdev.showdoc import show_doc
from fastcore.all import *
import shutil

## nbdev 1 --> 2

#| hide

### Add Alias Path To Avoid Broken Links - 

When migrating from fastpages which was Jekyll based to Quarto, we want to construct aliases to mitigate broken links.  

In [ ]:
#| hide
# These are just utilities for testing
def _get_fm(path):
    "A test utility that is used to get front matter"
    nbp = NBProcessor(path, procs=FrontmatterProc)
    nbp.process()
    return nbp.nb.frontmatter_

def _get_raw_fm(nb): 
    return first(L(nb.cells).filter(lambda x: x.cell_type == 'raw')).source

In [ ]:
#| export
def _cat_slug(fmdict):
    "Get the partial slug from the category front matter."
    slug = '/'.join(fmdict.get('categories', ''))
    return '/' + slug if slug else '' 

In [ ]:
#| hide
_fm1 = _get_fm('../../tests/2020-09-01-fastcore.ipynb')
test_eq(_cat_slug(_fm1), '/fastcore/fastai')

_fm2 = _get_fm('../../tests/2020-02-20-test.ipynb')
test_eq(_cat_slug(_fm2), '/jupyter')

In [ ]:
#| export
def _file_slug(fname): 
    "Get the partial slug from the filename."
    p = Path(fname)
    dt = '/'+p.name[:10].replace('-', '/')+'/'
    return dt + p.stem[11:]    

In [ ]:
#| hide
test_eq(_file_slug('../../tests/2020-09-01-fastcore.ipynb'), 
        '/2020/09/01/fastcore')

In [ ]:
#| export
def _replace_fm(d:dict, # dictionary you wish to conditionally change
                k:str,  # key to check 
                val:str,# value to check if d[k] == v
                repl_dict:dict #dictionary that will be used as a replacement 
               ):
    "replace key `k` in dict `d` if d[k] == val with `repl_dict`"
    if str(d.get(k, '')).lower().strip() == str(val.lower()).strip():
        d.pop(k)
        d = merge(d, repl_dict)
    return d

def _fp_fm(d):
    "create aliases for fastpages front matter to match Quarto front matter."
    d = _replace_fm(d, 'search_exclude', 'true', {'search':'false'})
    d = _replace_fm(d, 'hide', 'true', {'draft': 'true'})
    return d

In [ ]:
#| export
def _fp_image(d):
    "Correct path of fastpages images to reference the local directory."
    prefix = 'images/copied_from_nb/'
    if d.get('image', '').startswith(prefix): d['image'] = d['image'].replace(prefix, '')
    return d

In [ ]:
#| export
def _rm_quote(s): 
    title = re.search('''"(.*?)"''', s)
    return title.group(1) if title else s

def _is_jekyll_post(path): return bool(re.search(r'^\d{4}-\d{2}-\d{2}-', Path(path).name))

def _fp_convert(fm:dict, path:Path):
    "Make fastpages frontmatter Quarto complaint and add redirects."
    fs = _file_slug(path)
    if _is_jekyll_post(path):
        fm = compose(_fp_fm, _fp_image)(fm)
        if 'permalink' in fm: fm['aliases'] = [f"{fm['permalink'].strip()}"]
        else: fm['aliases'] = [f'{_cat_slug(fm) + fs}']
        if not fm.get('date'): 
            _,y,m,d,_ = fs.split('/')
            fm['date'] = f'{y}-{m}-{d}'
        
    if fm.get('summary') and not fm.get('description'): fm['description'] = fm['summary']
    if fm.get('tags') and not fm.get('categories'): 
        if isinstance(fm['tags'], str): fm['categories'] = fm['tags'].split()
        elif isinstance(fm['tags'], list): fm['categories'] = fm['tags']
    for k in ['title', 'description']:
        if k in fm: fm[k] = _rm_quote(fm[k])
    if fm.get('comments'): fm.pop('comments') #true by itself is not a valid value for comments https://quarto.org/docs/output-formats/html-basics.html#commenting, and the default is true
    return fm

In [ ]:
#| hide
_fm1 = _fp_convert(_fm1, '../../tests/2020-09-01-fastcore.ipynb')
test_eq(_fm1['aliases'], ['/fastcore/'])
test_eq(_fm1['image'], 'fastcore_imgs/td.png')

assert 'hide' not in _fm1 and 'draft' in _fm1
assert 'search_exclude' not in _fm1 and 'search' in _fm1
assert 'comments' not in _fm1

_fm2 = _fp_convert(_fm2, '../../tests/2020-02-20-test.ipynb')
test_eq(_fm2['aliases'], ['/jupyter/2020/02/20/test'])

### Migrate notebooks

In [ ]:
#| export
class MigrateProc(Processor):
    "Migrate fastpages front matter in notebooks to a raw cell."
    def begin(self): 
        self.nb.frontmatter_ = _fp_convert(self.nb.frontmatter_, self.nb.path_)
        if getattr(first(self.nb.cells), 'cell_type', None) == 'raw': del self.nb.cells[0]
        _insertfm(self.nb, self.nb.frontmatter_)

Before you migrate the fastpages notebook, the front matter is specified in Markdown like this:

In [ ]:
_tst_nb = '../../tests/2020-09-01-fastcore.ipynb'
print(read_nb(_tst_nb).cells[0].source)

# "fastcore: An Underrated Python Library"

> A unique python library that extends the python programming language and provides utilities that enhance productivity.
- author: "<a href='https://twitter.com/HamelHusain'>Hamel Husain</a>"
- toc: false
- image: images/copied_from_nb/fastcore_imgs/td.png
- comments: true
- search_exclude: true
- hide: true
- categories: [fastcore, fastai]
- permalink: /fastcore/
- badges: true


After migrating the notebook, the front matter is moved to a raw cell, and some of the fields are converted to be compliant with Quarto.  Furthermore, aliases may be added in order to prevent broken links:

In [ ]:
nbp = NBProcessor('../../tests/2020-09-01-fastcore.ipynb', procs=[FrontmatterProc, MigrateProc])
nbp.process()
_fm1 = _get_raw_fm(nbp.nb)
print(_fm1)

---
aliases:
- /fastcore/
author: <a href='https://twitter.com/HamelHusain'>Hamel Husain</a>
badges: true
categories:
- fastcore
- fastai
date: '2020-09-01'
description: A unique python library that extends the python programming language
  and provides utilities that enhance productivity.
draft: 'true'
image: fastcore_imgs/td.png
output-file: 2020-09-01-fastcore.html
permalink: /fastcore/
search: 'false'
title: 'fastcore: An Underrated Python Library'
toc: false

---




In [ ]:
#| hide
test_eq(_fm1, """---
aliases:
- /fastcore/
author: <a href='https://twitter.com/HamelHusain'>Hamel Husain</a>
badges: true
categories:
- fastcore
- fastai
date: '2020-09-01'
description: A unique python library that extends the python programming language
  and provides utilities that enhance productivity.
draft: 'true'
image: fastcore_imgs/td.png
output-file: 2020-09-01-fastcore.html
permalink: /fastcore/
search: 'false'
title: 'fastcore: An Underrated Python Library'
toc: false

---

""")


_res="""---
aliases:
- /jupyter/2020/02/20/test
badges: true
categories:
- jupyter
date: '2020-02-20'
description: A tutorial of fastpages for Jupyter notebooks.
image: images/chart-preview.png
output-file: 2020-02-20-test.html
title: Fastpages Notebook Blog Post
toc: true

---

"""

nbp = NBProcessor('../../tests/2020-02-20-test.ipynb', procs=[FrontmatterProc, MigrateProc])
nbp.process()
_fm2 = _get_raw_fm(nbp.nb)
test_eq(_fm2, _res)

### Migrate Fastpages Markdown Front Matter

In [ ]:
#| export
def fp_md_fm(path):
    "Make fastpages front matter in markdown files quarto compliant."
    p = Path(path)
    md = p.read_text()
    fm = _fm2dict(md, nb=False)
    if fm:
        fm = _fp_convert(fm, path)
        return _re_fm_md.sub(_dict2fm(fm), md)
    else: return md 

Here is what the front matter of a fastpages markdown post looks like before migration:

In [ ]:
#| eval: false
print(run('head -n13 ../../tests/2020-01-14-test-markdown-post.md'))

---

toc: true
layout: post
description: A minimal example of using markdown with fastpages.
categories: [markdown]
title: An Example Markdown Post


---

# Example Markdown Post


And this is what it looks like after migration:

In [ ]:
_res = fp_md_fm('../../tests/2020-01-14-test-markdown-post.md')
print(_res[:300])

---
aliases:
- /markdown/2020/01/14/test-markdown-post
categories:
- markdown
date: '2020-01-14'
description: A minimal example of using markdown with fastpages.
layout: post
title: An Example Markdown Post
toc: true

---

# Example Markdown Post

## Basic setup

Jekyll requires blog post files to b


In [ ]:
#| hide
assert """---
aliases:
- /markdown/2020/01/14/test-markdown-post
categories:
- markdown
date: '2020-01-14'
description: A minimal example of using markdown with fastpages.
layout: post
title: An Example Markdown Post
toc: true

---

""" in _res

In [ ]:
#hide
_res = fp_md_fm('../../tests/2022-09-06-homeschooling.md')
test_eq(_res,
"""---
aliases:
- /2022/09/06/homeschooling
author: Rachel Thomas
categories:
- advice
- health
date: '2022-09-06'
description: You can permanently damage your back, neck, and wrists from working without
  an ergonomic setup.  Learn how to create one for less at home.
image: /images/ergonomic1-short.jpg
summary: You can permanently damage your back, neck, and wrists from working without
  an ergonomic setup.  Learn how to create one for less at home.
tags: advice health
title: 'Essential Work-From-Home Advice: Cheap and Easy Ergonomic Setups'

---

Lorem ipsum
""")

### Directives

nbdev v2 directives start with a `#|` whereas v1 directives were comments without a pipe `|`.

In [ ]:
#| export
_alias = merge({k:'code-fold: true' for k in ['collapse', 'collapse_input', 'collapse_hide']}, 
               {'collapse_show':'code-fold: show', 'hide_input': 'echo: false', 'hide': 'include: false', 'hide_output': 'output: false'})
def _subv1(s): return _alias.get(s, s)

In [ ]:
#| export
def _re_v1():
    d = ['default_exp', 'export', 'exports', 'exporti', 'hide', 'hide_input', 'collapse_show', 'collapse',
         'collapse_hide', 'collapse_input', 'hide_output',  'default_cls_lvl']
    d += L(get_config().tst_flags).filter()
    d += [s.replace('_', '-') for s in d] # allow for hyphenated version of old directives
    _tmp = '|'.join(list(set(d)))
    return re.compile(fr"^[ \f\v\t]*?(#)\s*({_tmp})(?!\S)", re.MULTILINE)

def _repl_directives(code_str): 
    def _fmt(x): return f"#| {_subv1(x[2].replace('-', '_').strip())}"
    return _re_v1().sub(_fmt, code_str)

In [ ]:
_test_dir = """
#default_exp
 #export
# collapse-show
#collapse-hide
#collapse
# collapse_output
not_dir='#export'
# hide_input
foo
# hide
"""
test_eq(_repl_directives(_test_dir),
"""
#| default_exp
#| export
#| code-fold: show
#| code-fold: true
#| code-fold: true
# collapse_output
not_dir='#export'
#| echo: false
foo
#| include: false
""")

In [ ]:
#| export
def _repl_v1dir(cell):
    "Replace nbdev v1 with v2 directives."
    if cell.get('source') and cell.get('cell_type') == 'code':
        ss = cell['source'].splitlines()
        first_code = first_code_ln(ss, re_pattern=_re_v1())
        if not first_code: first_code = len(ss)
        if not ss: pass
        else: cell['source'] = '\n'.join([_repl_directives(c) for c in ss[:first_code]] + ss[first_code:])

In [ ]:
show_doc(_repl_v1dir)

---

[source](https://github.com/AnswerDotAI/nbdev/blob/main/nbdev/migrate.py#L122){target="_blank" style="float:right; font-size:smaller"}

### _repl_v1dir

```python

def _repl_v1dir(
    cell
):


```

*Replace nbdev v1 with v2 directives.*

for example, if any of the lines below are valid nbdev v1 directives, they replaced with a `#|`, but only before the first line of code:

In [ ]:
#| hide
_code = _test_dir

tst = {'cell_type': 'code', 'execution_count': 26,
       'metadata': {'hide_input': True, 'meta': 23},
       'outputs': [{'execution_count': 2,
                    'data': {
                        'application/vnd.google.colaboratory.intrinsic+json': {'type': 'string'},
                        'plain/text': ['sample output',]
                    }, 'output': 'super'}],
       'source': _code}
nb = {'metadata': {'kernelspec': 'some_spec', 'jekyll': 'some_meta', 'meta': 37}, 'cells': [tst]}

for cell in nb['cells']: _repl_v1dir(cell) 

test_eq(nb['cells'][0]['source'], 
"""
#| default_exp
#| export
#| code-fold: show
#| code-fold: true
#| code-fold: true
# collapse_output
not_dir='#export'
# hide_input
foo
# hide""")

### Callouts

In fastpages, there was a markdown shortuct for callouts for `Note`, `Tip`, `Important` and `Warning` with block quotes (these only worked in notebooks).  Since Quarto has its own [callout blocks](https://quarto.org/docs/authoring/callouts.html#callout-types) with markdown syntax, we do not implement these shortcuts in nbdev.  Instead, we offer a manual conversion utility for these callouts so that you can migrate from fastpages to Quarto.

In [ ]:
#| export
_re_callout = re.compile(r'^>\s(Warning|Note|Important|Tip):(.*)', flags=re.MULTILINE)
def _co(x): return ":::{.callout-"+x[1].lower()+"}\n\n" + f"{x[2].strip()}\n\n" + ":::"
def _convert_callout(s): 
    "Convert nbdev v1 to v2 callouts."
    return _re_callout.sub(_co, s)

In [ ]:
show_doc(_convert_callout)

---

[source](https://github.com/AnswerDotAI/nbdev/blob/main/nbdev/migrate.py#L134){target="_blank" style="float:right; font-size:smaller"}

### _convert_callout

```python

def _convert_callout(
    s
):


```

*Convert nbdev v1 to v2 callouts.*

For example, the below markdown:

In [ ]:
_callouts="""
## Boxes / Callouts

> Warning: There will be no second warning!

Other text

> Important: Pay attention! It's important.

> Tip: This is my tip.

> Note: Take note of `this.`
"""

Gets converted to:

In [ ]:
#| echo:false
_c = _convert_callout(_callouts)
assert '> Tip:' not in _c
assert 'Other text' in _c
print(_c[:156])


## Boxes / Callouts

:::{.callout-warning}

There will be no second warning!

:::

Other text

:::{.callout-important}

Pay attention! It's important.

:::


### Video Embeds

In fastpages, you could embed videos with a simple markdown shortcut involving a block quote with the prefix `youtube:`, that looked like this

`> youtube: https://youtu.be/XfoYk_Z5AkI`

However, in Quarto you can use the [video extension](https://github.com/quarto-ext/video) to embed videos.


In [ ]:
#| export
_re_video = re.compile(r'^>\syoutube:(.*)', flags=re.MULTILINE)
def _v(x): return "{{< " + f"video {x[1].strip()}" + " >}}"
def _convert_video(s):
    "Replace nbdev v1 with v2 video embeds."
    return _re_video.sub(_v, s)

In [ ]:
show_doc(_convert_video)

---

[source](https://github.com/AnswerDotAI/nbdev/blob/main/nbdev/migrate.py#L141){target="_blank" style="float:right; font-size:smaller"}

### _convert_video

```python

def _convert_video(
    s
):


```

*Replace nbdev v1 with v2 video embeds.*

In [ ]:
_videos="""
## Videos

> youtube: https://youtu.be/XfoYk_Z5AkI
"""

In [ ]:
print(_convert_video(_videos))


## Videos

{{< video https://youtu.be/XfoYk_Z5AkI >}}



In [ ]:
#| export
_shortcuts = compose(_convert_video, _convert_callout)

def _repl_v1shortcuts(cell):
    "Replace nbdev v1 with v2 callouts."
    if cell.get('source') and cell.get('cell_type') == 'markdown':
        cell['source'] = _shortcuts(cell['source'])

In [ ]:
#| export
def migrate_nb(path, overwrite=True):
    "Migrate Notebooks from nbdev v1 and fastpages."
    nbp = NBProcessor(path, procs=[FrontmatterProc, MigrateProc, _repl_v1shortcuts, _repl_v1dir], rm_directives=False)
    nbp.process()
    if overwrite: write_nb(nbp.nb, path)
    return nbp.nb

In [ ]:
#| export
def migrate_md(path, overwrite=True):
    "Migrate Markdown Files from fastpages."
    txt = fp_md_fm(path)
    if overwrite: path.write_text(txt)
    return txt

In [ ]:
#| export
@call_parse
def nbdev_migrate(
    path:str=None, # A path or glob containing notebooks and markdown files to migrate
    no_skip:bool=False, # Do not skip directories beginning with an underscore
):
    "Convert all markdown and notebook files in `path` from v1 to v2"
    _skip_re = None if no_skip else '^[_.]'
    if path is None: path = get_config().nbs_path
    for f in globtastic(path, file_re='(.ipynb$)|(.md$)', skip_folder_re=_skip_re, func=Path):
        try:
            if f.name.endswith('.ipynb'): migrate_nb(f)
            if f.name.endswith('.md'): migrate_md(f)
        except Exception as e: raise Exception(f'Error in migrating file: {f}') from e

In [ ]:
#| hide

### Test Notebook Migration ####
_orig =  Path('../../tests/2020-02-20-test.ipynb') # nbdev v1 notebook
_tmp =  Path('../../tests/2020-02-20-test-COPY.ipynb') # A copy of this nb that will be migrated
shutil.copy(_orig, _tmp)
nbdev_migrate(_tmp)

def _nb2str(p): return str(L(read_nb(p).cells).attrgot('source'))
assert ':::{.callout-warning}' not in _nb2str(_orig) and ':::{.callout-warning}' in _nb2str(_tmp)
assert '#| code-fold: true' not in _nb2str(_orig) and '#| code-fold: true' in _nb2str(_tmp)
assert '#| single-value' not in _nb2str(_tmp)

if _tmp.exists(): _tmp.unlink() # missing_ok not in python 3.7

In [ ]:
#| hide

### Test Markdown Migration #####
_orig =  Path('../../tests/2020-01-14-test-markdown-post.md') # fastpages markdown
_tmp =  Path('../../tests/2020-01-14-test-markdown-post-COPY.md') # A copy to be migrated
shutil.copy(_orig, _tmp)
nbdev_migrate(_tmp)

assert _tmp.read_text().startswith("""---
aliases:
- /markdown/2020/01/14/test-markdown-post-COPY
categories:
- markdown
date: '2020-01-14'
description: A minimal example of using markdown with fastpages.
layout: post
title: An Example Markdown Post
toc: true

---""")

if _tmp.exists(): _tmp.unlink() # missing_ok not in python 3.7

## nbdev 2 --> 3

### Migrate settings.ini to pyproject.toml

In [ ]:
#| export
_license_map = {'apache2': 'Apache-2.0', 'mit': 'MIT', 'gpl2': 'GPL-2.0', 'gpl3': 'GPL-3.0', 'bsd3': 'BSD-3-Clause'}

In [ ]:
#| export
def _migrate_workflows(path):
    "Update GitHub workflow files to use nbdev3 workflows"
    wf_path = Path(path) / '.github/workflows'
    if not wf_path.exists(): return
    replacements = [
        ('fastai/workflows/quarto-ghp@', 'fastai/workflows/quarto-ghp3@'),
        ('fastai/workflows/nbdev-ci@', 'fastai/workflows/nbdev3-ci@'),
    ]
    for f in (*wf_path.glob('*.yml'), *wf_path.glob('*.yaml')):
        txt = f.read_text()
        for old, new in replacements: txt = txt.replace(old, new)
        f.write_text(txt)

In [ ]:
#| export
def _toml_val(v):
    if v.lower() in ('true','false'): return v.lower()
    return repr(v)
def _py_val(v):
    try: return str2bool(v)
    except: return v


def _fmt_script(s):
    name,val = s.strip().split('=')
    return f'{name.strip()} = "{val.strip()}"'


def _build_classifiers(d):
    "Build list of classifier strings from config dict"
    _status_map = {'1': 'Planning', '2': 'Pre-Alpha', '3': 'Alpha', '4': 'Beta', '5': 'Production/Stable', '6': 'Mature', '7': 'Inactive'}
    cs = ['Programming Language :: Python :: 3', 'Programming Language :: Python :: 3 :: Only']
    status,audience,language = d.get('status', '').strip(), d.get('audience', '').strip(), d.get('language', '').strip()
    if status and status in _status_map: cs.insert(0, f"Development Status :: {status} - {_status_map[status]}")
    if audience: cs.insert(0, f"Intended Audience :: {audience}")
    if language: cs.insert(0, f"Natural Language :: {language}")
    return cs

In [ ]:
#| export
def _nbdev_migrate_config(d, path):  # Config dict from settings.ini
    "Migrate settings.ini dict to pyproject.toml string"
    repo = d.get('repo', '')
    user = d.get('user', '')
    lib_path = d.get('lib_path', repo.replace('-', '_'))
    branch = d.get('branch', 'main')
    
    git_url = d.get('git_url') or (f"https://github.com/{user}/{repo}" if user else '')
    doc_host = d.get('doc_host', '')
    doc_baseurl = d.get('doc_baseurl', '')
    doc_url = (doc_host.rstrip('/') + doc_baseurl) if doc_host else (f"https://{user}.github.io/{repo}/" if user else '')
    set_version(path/lib_path, d.get('version', '0.0.1'))
    lib_name = d.get('lib_name', repo)
    txt = pyproject_tmpl.format(name=lib_name, lib_path=lib_path, description=d.get('description', ''),
        min_python=d.get('min_python', '3.10'), license=_license_map.get(d.get('license', ''), d.get('license', 'Apache-2.0')),
        author=d.get('author', ''), author_email=d.get('author_email', ''),
        keywords=d.get('keywords', 'nbdev').split(), git_url=git_url, doc_url=doc_url, branch=branch)
    # Add dependencies (combine requirements + pip_requirements)
    reqs = d.get('requirements', '').split() + d.get('pip_requirements', '').split()
    if reqs: txt = txt.replace('dependencies = []', f'dependencies = {reqs}')
    dev_reqs = d.get('dev_requirements', '').split()
    if dev_reqs: txt = txt.replace('[tool.setuptools', f'[project.optional-dependencies]\ndev = {dev_reqs}\n\n[tool.setuptools', 1)
    # Add console_scripts
    scripts = d.get('console_scripts', '').strip()
    if scripts:
        scripts_lines = [_fmt_script(s) for s in scripts.split('\n') if s.strip()]
        scripts_toml = '\n[project.scripts]\n' + '\n'.join(scripts_lines)
        txt = txt.replace('[tool.setuptools', scripts_toml + '\n\n[tool.setuptools', 1)
    _classifiers_default = 'classifiers = [\n    "Programming Language :: Python :: 3",\n    "Programming Language :: Python :: 3 :: Only",\n]'
    txt = txt.replace(_classifiers_default, 'classifiers = ' + repr(_build_classifiers(d)).replace("'", '"'))
    
    nbdev_settings = {k:d[k] for k in ('nbs_path','doc_path','branch','recursive','readme_nb','tst_flags',
        'clean_ids','clear_all','put_version_in_init','jupyter_hooks','custom_sidebar','title')
        if k in d and _py_val(d[k]) != nbdev_defaults.get(k) and not (k=='title' and d[k]==repo)}
    if nbdev_settings:
        nbdev_toml = '\n'.join(f'{k} = {_toml_val(v)}' for k,v in nbdev_settings.items())
        txt = txt.rstrip() + '\n' + nbdev_toml + '\n'
    _migrate_workflows(path)
    return txt

In [ ]:
#| export
@call_parse
def nbdev_migrate_config(path:str='.'):  # Project root containing settings.ini
    "Migrate settings.ini to pyproject.toml"
    path = Path(path)
    ini = path/'settings.ini'
    if not ini.exists(): return print(f"No settings.ini found at {ini}")
    cfg = Config(path, 'settings.ini')
    txt = _nbdev_migrate_config(cfg.d, path)
    (path/'pyproject.toml').write_text(txt)
    print(f"Created {path/'pyproject.toml'}. You can now delete {ini} and setup.py (if present)")

In [ ]:
import tempfile

In [ ]:
with tempfile.TemporaryDirectory() as d:
    # Create a minimal settings.ini
    (Path(d)/'settings.ini').write_text('''[DEFAULT]
repo = test-proj
user = testuser
author = Test Author
author_email = test@test.com
description = A test project
version = 1.0.0
''')
    nbdev_migrate_config(d)
    assert (Path(d)/'pyproject.toml').exists()
    txt = (Path(d)/'pyproject.toml').read_text()
    assert 'name = "test-proj"' in txt
    assert '[tool.nbdev]' in txt
    assert (Path(d)/'test_proj/__init__.py').exists()

Created /var/folders/51/b2_szf2945n072c0vj2cyty40000gn/T/tmpajhht0am/pyproject.toml. You can now delete /var/folders/51/b2_szf2945n072c0vj2cyty40000gn/T/tmpajhht0am/settings.ini and setup.py (if present)


In [ ]:
_test_settings_ini = '''[DEFAULT]
repo = fasthtml
lib_name = python-fasthtml
version = 0.12.40
min_python = 3.10
license = apache2
requirements = fastcore>=1.10.0 python-dateutil starlette>0.33 oauthlib itsdangerous uvicorn[standard]>=0.30
dev_requirements = ipython lxml pysymbol_llm monsterui PyJWT
black_formatting = False
conda_user = fastai
doc_path = _docs
lib_path = fasthtml
nbs_path = nbs
recursive = True
tst_flags = notest
put_version_in_init = True
branch = main
custom_sidebar = False
doc_host = https://www.fastht.ml
doc_baseurl = /docs/
git_url = https://github.com/AnswerDotAI/fasthtml
title = fasthtml
audience = Developers
author = Jeremy Howard and contributors
author_email = github@jhoward.fastmail.fm
copyright = 2024 onwards, Jeremy Howard
description = The fastest way to create an HTML app
keywords = nbdev jupyter notebook python
language = English
status = 3
console_scripts = fh_railway_link=fasthtml.cli:railway_link
	fh_railway_deploy=fasthtml.cli:railway_deploy
user = AnswerDotAI
readme_nb = index.ipynb
allowed_metadata_keys = 
allowed_cell_metadata_keys = 
jupyter_hooks = True
clean_ids = True
clear_all = False
cell_number = False
skip_procs = 
update_pyproject = True
'''

In [ ]:
from configparser import ConfigParser
try: import tomllib
except ImportError: import tomli as tomllib

In [ ]:
cfg = ConfigParser()
cfg.read_string(_test_settings_ini)
ini = dict2obj(dict(cfg['DEFAULT']))
first(ini.items())

('repo', 'fasthtml')

In [ ]:
with tempfile.TemporaryDirectory() as d:
    result = _nbdev_migrate_config(dict(cfg['DEFAULT']), Path(d))
    toml = tomllib.loads(result)
proj, nbdev = toml['project'], nested_idx(toml, 'tool', 'nbdev')
proj['name']

'python-fasthtml'

In [ ]:
proj = toml['project']
test_eq(first(nested_idx(proj, 'entry-points', 'nbdev')), ini.repo)
assert ini.repo in nested_idx(proj, 'urls', 'Repository')

In [ ]:
test_eq(proj['name'], ini.lib_name)
test_eq(nested_idx(toml, 'tool', 'setuptools', 'dynamic', 'version', 'attr'), f"{ini.lib_path}.__version__")
test_eq(proj['requires-python'], f">={ini.min_python}")
test_eq(nested_idx(proj, 'license', 'text'), 'Apache-2.0')  # 'apache2' maps to 'Apache-2.0'
test_eq(proj['dependencies'], ini.requirements.split())
test_eq(nested_idx(proj, 'optional-dependencies', 'dev'), ini.dev_requirements.split())
test_eq(nested_idx(toml, 'tool', 'setuptools', 'packages', 'find', 'include'), [ini.lib_path])
test_eq(nested_idx(proj, 'urls', 'Documentation'), f"{ini.doc_host}{ini.doc_baseurl}")
test_eq(nested_idx(proj, 'urls', 'Repository'), ini.git_url)
nbdev = nested_idx(toml, 'tool', 'nbdev')
test_eq(nbdev['jupyter_hooks'], ini.jupyter_hooks.lower() == 'true')
test_eq(nbdev['custom_sidebar'], ini.custom_sidebar.lower() == 'true')
test_eq(nested_idx(proj, 'authors', 0, 'name'), ini.author)
test_eq(nested_idx(proj, 'authors', 0, 'email'), ini.author_email)
test_eq(proj['description'], ini.description)
test_eq(proj['keywords'], ini.keywords.split())
test_eq(nested_idx(proj, 'scripts', 'fh_railway_link'), 'fasthtml.cli:railway_link')

In [ ]:
scripts = ini.console_scripts.strip().split('\n')
for s in scripts:
    name,val = s.split('=')
    test_eq(nested_idx(proj, 'scripts', name.strip()), val.strip())

assert f"Development Status :: {ini.status}" in str(proj['classifiers'])
assert f"Intended Audience :: {ini.audience}" in str(proj['classifiers'])
assert f"Natural Language :: {ini.language}" in str(proj['classifiers'])

assert ini.user in nested_idx(proj, 'urls', 'Repository')

In [ ]:
from collections import Counter

In [ ]:
def find_dupes(d, path=''):
    res = {}
    for k,v in (d.items() if isinstance(d, dict) else enumerate(d) if isinstance(d, list) else []):
        p = f"{path}.{k}" if path else str(k)
        if isinstance(v, dict): res.update(find_dupes(v, p))
        elif isinstance(v, list):
            hashable = [x for x in v if isinstance(x, (str, int, float, bool, tuple))]
            counts = Counter(hashable)
            dupes = {x:n for x,n in counts.items() if n > 1}
            if dupes: res[p] = dupes
    return res

test_eq(find_dupes(toml), {})

## Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()